In [1]:
from collections import defaultdict


facts = {
    "American(Robert)",
    "Enemy(Nono, America)",
    "Owns(Nono, T1)",
    "Missile(T1)"
}

rules = [
    ("Missile(x) => Weapon(x)"),
    ("Owns(Nono, x) ^ Missile(x) => Sells(Robert, x, Nono)"),
    ("Enemy(x, America) => Hostile(x)"),
    ("American(p) ^ Weapon(q) ^ Sells(p, q, r) ^ Hostile(r) => Criminal(p)")
]

query = "Criminal(Robert)"




def substitute(expr, var, const):
    return expr.replace(f"({var})", f"({const})").replace(f",{var}", f",{const}")

def forward_chaining(facts, rules, query):
    inferred = set()
    while True:
        new_fact_added = False
        for rule in rules:
            lhs, rhs = rule.split("=>")
            premises = [p.strip() for p in lhs.split("^")]
            conclusion = rhs.strip()

           
            variable_bindings = {}

            for fact in facts:
                for premise in premises:
                    if "(" in premise and "(" in fact:
                        pname = premise.split("(")[0]
                        fname = fact.split("(")[0]
                        if pname == fname:
                            p_args = premise[premise.find("(")+1:-1].split(",")
                            f_args = fact[fact.find("(")+1:-1].split(",")
                            for pa, fa in zip(p_args, f_args):
                                if pa[0].islower():  # variable
                                    variable_bindings[pa.strip()] = fa.strip()

            
            new_fact = conclusion
            for var, val in variable_bindings.items():
                new_fact = substitute(new_fact, var, val)

          
            if all(any(pname.split("(")[0] == f.split("(")[0] for f in facts) for pname in premises):
                if new_fact not in facts:
                    facts.add(new_fact)
                    inferred.add(new_fact)
                    new_fact_added = True
                    print(f"Inferred new fact: {new_fact}")

                    if new_fact == query:
                        print(f"\n✅ Query {query} proven by Forward Chaining!")
                        return True

        if not new_fact_added:
            break

    print(f"\n❌ Query {query} could not be proven.")
    return False



print("Initial Facts:", facts)
print("\nApplying Forward Chaining Rules...\n")
forward_chaining(facts, rules, query)


Initial Facts: {'Owns(Nono, T1)', 'Enemy(Nono, America)', 'American(Robert)', 'Missile(T1)'}

Applying Forward Chaining Rules...

Inferred new fact: Weapon(T1)
Inferred new fact: Sells(Robert, x, Nono)
Inferred new fact: Hostile(Nono)
Inferred new fact: Criminal(Robert)

✅ Query Criminal(Robert) proven by Forward Chaining!


True